<a href="https://colab.research.google.com/github/Natural-State/agol-data-workflows/blob/master/code/Colab%20notebooks/16_SPL4SMGP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Extract Soil moisture measure from SMAP Level-4 platform

## Import gee and authenticate

In [ ]:
import os

# If credentials file doesn't exist, authenticate and store credentials
# Else if credentials file does exist, use stored credentials and initialise
if not os.path.exists(os.path.expanduser("~/.config/earthengine/credentials")):
  import ee
  ee.Authenticate()
  ee.Initialize()
else:
  import ee
  ee.Initialize()

## Input arguments for data extraction

In [ ]:
# Area of interest
aoi = ee.FeatureCollection("projects/ns-agol-rs-data/assets/MKR_NS_buff_5km")
aoi_name = "MKR_NS_buff_5km"

# GEE layer ID
layer_dict = {
  "sm_surface": "RS_049",
  "sm_rootzone": "RS_050",
  "sm_surface_wetness": "RS_051",
  "sm_rootzone_wetness": "RS_052"
}

# Date parameters
start_year = 2018
end_year = 2022

# Range doesn't include the stop value
year_list = ee.List(list(range(start_year, end_year+1)))

# Season parameters (months)
rain_start = 3
rain_end = 5
dry_start = 7
dry_end = 10

# Image reducer (options: mean, median, min, max, stdDev, sum, product)
img_col_reducer = "mean"

## SMAP L4 Global 3-hourly 9 km EASE-Grid Surface and Root Zone Soil Moisture Geophysical Data, Version 7


In [ ]:
smap = ee.ImageCollection("NASA/SMAP/SPL4SMGP/007") \
  .filterBounds(aoi)

## Create reducer list

In [ ]:
reducer_list = ee.Reducer.mean() \
.combine(reducer2 = ee.Reducer.median(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.min(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.max(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.stdDev(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.sum(), sharedInputs=True) \
.combine(reducer2 = ee.Reducer.product(), sharedInputs=True)

## Soil moisture processing

In [ ]:
# Function to get mean soil moisture values for each year and season

def annual_seasonal_mean(soil_band, year_date, season_start, season_end):
  start = ee.Date.fromYMD(year_date, season_start, 1)
  end = ee.Date.fromYMD(year_date, season_end, 30)
  date_range = ee.DateRange(start, end)
  return smap \
        .filterDate(date_range) \
        .select(soil_band) \
        .reduce(reducer = reducer_list) \
        .clip(aoi) \
        .set({'year': year_date,
             'season_start': season_start,
             'season_end': season_end})

#
def map_annual_1(year_date):
  return annual_seasonal_mean("sm_surface", year_date, rain_start, rain_end)

annual_surf_wet = year_list.map(map_annual_1)

#
def map_annual_2(year_date):
  return annual_seasonal_mean("sm_surface", year_date, dry_start, dry_end)

annual_surf_dry = year_list.map(map_annual_2)

#
def map_annual_3(year_date):
  return annual_seasonal_mean("sm_rootzone", year_date, rain_start, rain_end)

annual_root_wet = year_list.map(map_annual_3)

#
def map_annual_4(year_date):
  return annual_seasonal_mean("sm_rootzone", year_date, dry_start, dry_end)

annual_root_dry = year_list.map(map_annual_4)

#
def map_annual_5(year_date):
  return annual_seasonal_mean("sm_surface_wetness", year_date, rain_start, rain_end)

annual_surfWI_wet = year_list.map(map_annual_5)

#
def map_annual_6(year_date):
  return annual_seasonal_mean("sm_surface_wetness", year_date, dry_start, dry_end)

annual_surfWI_dry = year_list.map(map_annual_6)

#
def map_annual_7(year_date):
  return annual_seasonal_mean("sm_rootzone_wetness", year_date, rain_start, rain_end)

annual_rootWI_wet = year_list.map(map_annual_7)

#
def map_annual_8(year_date):
  return annual_seasonal_mean("sm_rootzone_wetness", year_date, dry_start, dry_end)

annual_rootWI_dry = year_list.map(map_annual_8)

## Output checks

In [ ]:
## Check an element of list
year_mosaic = ee.Image(annual_surf_wet.get(0))
label = ee.String(year_mosaic.get('year')).getInfo()
print(label)
print(year_mosaic.getInfo())
print(year_mosaic.bandNames().getInfo())

## Check a reducer band
band_select = ".*" + img_col_reducer
print(band_select)
print(year_mosaic.select(band_select).getInfo())

## Check an element of list
year_mosaic = ee.Image(annual_surfWI_dry.get(0))
label = ee.String(year_mosaic.get('year')).getInfo()
print(label)
print(year_mosaic.getInfo())
print(year_mosaic.bandNames().getInfo())

## Check a reducer band
band_select = ".*" + img_col_reducer
print(band_select)
print(year_mosaic.select(band_select).getInfo())

## Export data - create task

In [ ]:
def export_layers(annual_soil_col, layer_name):

  for i in  range(ee.List.length(annual_soil_col).getInfo()):
    band_select = ".*" + img_col_reducer
    output_img =  ee.Image(annual_soil_col.get(i))
    output_img = output_img.select(band_select)
    year_label = ee.String(output_img.get('year')).getInfo()
    season =  ee.String(output_img.get('season_start')).getInfo()
    season_label = "dry" if season == dry_start else "wet"
    output_name = f"{layer_name}_{img_col_reducer}_{aoi_name}_{ee.String(output_img.get('year')).getInfo()}_{season_label}"

    task = ee.batch.Export.image.toDrive(image = output_img,
                                        region = aoi.geometry(),
                                        description = "EXPORT IMAGE TO DRIVE",
                                        folder = "GEE_exports",
                                        fileNamePrefix = output_name,
                                        scale = 10000,
                                        maxPixels = 10e12
                                      )
    task.start()
    print("STARTED TASK ", i+1, layer_name)

In [ ]:
export_layers(annual_surf_wet, "RS_049")
export_layers(annual_surf_dry, "RS_049")
export_layers(annual_root_wet, "RS_050")
export_layers(annual_root_dry, "RS_050")

export_layers(annual_surfWI_wet, "RS_051")
export_layers(annual_surfWI_dry, "RS_051")
export_layers(annual_rootWI_wet, "RS_052")
export_layers(annual_rootWI_dry, "RS_052")

In [ ]:
# Print the list of variable/object names
variable_names = [var for var in dir() if not var.startswith('_')]
for v in variable_names:
  print(v)

## Check task status

[List](https://developers.google.com/earth-engine/guides/processing_environments#list-of-task-states) of task status messages (state field)


In [ ]:
tasks = ee.batch.Task.list()
for task in tasks[0:ee.List.length(year_list).getInfo()]:
  task_id = task.status()['id']
  task_state = task.status()['state']
  print(task_id, task_state)